In [77]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Load Data

In [86]:
X = np.load("X_feat_mat.npy")
y = np.load("y_label_vect.npy")

In [92]:
print(X.shape, y.shape)

(90, 12) (90,)


# Dimensionality Reduction

In [165]:
# Parameter
var_desired = .99
# SVD
svd = TruncatedSVD(n_components=X.shape[1]-1, n_iter=7, random_state=42)
svd.fit(X)
 
var_sum = svd.explained_variance_ratio_.sum()
var = 0
for n, v in enumerate(svd.explained_variance_ratio_):
    var += v
    if var/var_sum >= var_desired:
        print(var/var_sum)
        svd_trans = TruncatedSVD(n_components=n+1, n_iter=7, random_state=42)
        X_reduced = svd_trans.fit_transform(X)
        break

1.0


In [166]:
X_reduced.shape

(90, 3)

# Classifier

## - SciKit Random Forest

In [122]:
from sklearn.ensemble import RandomForestClassifier

In [135]:
clf_rf = RandomForestClassifier(n_estimators=10)
clf_rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [136]:
y_ = clf_rf.predict(X)
sum(y-y_) / len(y)

-0.5

## - SciKit Logistic Regression

In [138]:
from sklearn.linear_model import LogisticRegression

In [139]:
clf_lr = LogisticRegression()
clf_lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [140]:
y_ = clf_lr.predict(X)
sum(y-y_) / len(y)

0.5

## - SciKit Linear Discriminant Analysis